## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition','Department']]

In [4]:
# Create a list of at least 10 column names to use as X data
X_columns = [
    'Education',
    'Age',
    'DistanceFromHome',
    'JobSatisfaction',
    'OverTime',
    'StockOptionLevel',
    'WorkLifeBalance',
    'YearsAtCompany',
    'YearsSinceLastPromotion',
    'NumCompaniesWorked',
]

# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
X_df.dtypes

Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [6]:
X_df['OverTime'].value_counts()

OverTime
No     1054
Yes     416
Name: count, dtype: int64

In [8]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

from sklearn.preprocessing import OneHotEncoder

overtime_encoder = OneHotEncoder(sparse_output=False,drop='first')

overtime_encoder.fit(np.array(X_train["OverTime"]).reshape(-1,1))

X_train["OverTime_Encoded"]= overtime_encoder.transform(np.array(X_train["OverTime"]).reshape(-1,1))
X_test["OverTime_Encoded"]= overtime_encoder.transform(np.array(X_test["OverTime"]).reshape(-1,1))

X_train=X_train.drop(columns="OverTime")
X_test=X_test.drop(columns="OverTime")


In [9]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:

# Create a OneHotEncoder for the Department column

department_encoder = OneHotEncoder(sparse_output=False,drop='first')

# Fit the encoder to the training data
department_encoder.fit(np.array(y_train["Department"]).reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data
y_department_train= department_encoder.transform(np.array(y_train["Department"]).reshape(-1,1))
y_department_test= department_encoder.transform(np.array(y_test["Department"]).reshape(-1,1))


In [13]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False,drop='first')

# Fit the encoder to the training data
attrition_encoder.fit(np.array(y_train["Attrition"]).reshape(-1,1))

# Create two new variables by applying the encoder
# to the training and testing data

y_attrition_train= attrition_encoder.transform(np.array(y_train["Attrition"]).reshape(-1,1))
y_attrition_test= attrition_encoder.transform(np.array(y_test["Attrition"]).reshape(-1,1))

## Part 2: Create, Compile, and Train the Model

In [14]:
# Find the number of columns in the X training data.
X_train_features = len(X_train.columns)

# Create the input layer
input = layers.Input(shape=(X_train_features,),name="input")

# Create at least two shared layers

shared_1 = layers.Dense(64,activation="relu",name="shared_1")(input)
shared_2 = layers.Dense(128,activation="relu",name="shared_2")(shared_1)


In [15]:
# Create a branch for Department
# with a hidden layer and an output layer


# Create the hidden layer
department_hidden = layers.Dense(32,activation="relu",name="department_hidden")(shared_2)

# Create the output layer
department_output = layers.Dense(y_department_train.shape[1],activation="softmax",name="department_output")(department_hidden)

In [16]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden = layers.Dense(32,activation="relu",name="attrition_hidden")(shared_2)

# Create the output layer
attrition_output = layers.Dense(y_attrition_train.shape[1],activation="sigmoid",name="attrition_output")(attrition_hidden)

In [17]:
# Create the model

model = Model(inputs= input, outputs={"department_output": department_output, "attrition_output": attrition_output})

# Compile the model
model.compile(optimizer="adam", 
              loss={"department_output":"categorical_crossentropy",
                    "attrition_output":"binary_crossentropy"},
               metrics={"department_output": "accuracy",
                        "attrition_output": "accuracy"})

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 10)]         0           []                               
                                                                                                  
 shared_1 (Dense)               (None, 64)           704         ['input[0][0]']                  
                                                                                                  
 shared_2 (Dense)               (None, 128)          8320        ['shared_1[0][0]']               
                                                                                                  
 attrition_hidden (Dense)       (None, 32)           4128        ['shared_2[0][0]']               
                                                                                              

In [18]:
# Train the model
model.fit(X_train_scaled,{"department_output": y_department_train, "attrition_output": y_attrition_train},epochs=100)

Epoch 1/100


2025-02-06 17:45:20.151349: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


37/37 [==============================] - 0s 870us/step - loss: 1.1060 - attrition_output_loss: 0.4940 - department_output_loss: 0.6120 - attrition_output_accuracy: 0.8129 - department_output_accuracy: 0.6905
Epoch 2/100
37/37 [==============================] - 0s 701us/step - loss: 1.0090 - attrition_output_loss: 0.4129 - department_output_loss: 0.5962 - attrition_output_accuracy: 0.8359 - department_output_accuracy: 0.6930
Epoch 3/100
37/37 [==============================] - 0s 798us/step - loss: 0.9765 - attrition_output_loss: 0.3879 - department_output_loss: 0.5886 - attrition_output_accuracy: 0.8461 - department_output_accuracy: 0.6922
Epoch 4/100
37/37 [==============================] - 0s 881us/step - loss: 0.9580 - attrition_output_loss: 0.3720 - department_output_loss: 0.5860 - attrition_output_accuracy: 0.8486 - department_output_accuracy: 0.6930
Epoch 5/100
37/37 [==============================] - 0s 901us/step - loss: 0.9458 - attrition_output_loss: 0.3608 - department_outpu

In [23]:
# Evaluate the model with the testing data
model_results = model.evaluate(X_test_scaled,{"department_output":y_department_test, "attrition_output": y_attrition_test})

model_results

10/10 [==============================] - 0s 685us/step - loss: 706.8660 - attrition_output_loss: 0.9647 - department_output_loss: 705.9013 - attrition_output_accuracy: 0.8741 - department_output_accuracy: 0.7109


[706.8660278320312,
 0.9647217988967896,
 705.9013061523438,
 0.8741496801376343,
 0.7108843326568604]

In [22]:
# Print the accuracy for both department and attrition
print(model_results[3])
print(model_results[4])


0.8741496801376343
0.7108843326568604


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?



1. Accuracy is not the best metric to use on this data because we're dealing with imbalanced data, which means this could cause over/under sampling.
2. I chose softmax for the department data, as it yielded better results for the multiclass. For attrition, I used sigmoid as the attrition data is binary.
3. This model can be improved with more data, more layers- particularly the overlay of economic data. We only used 10 columns of data here, so the sample wasn't large enough for a real company to use this model in practice.